# Web Assessment
- Author: Aidan Marlin
- Date: 20221005
- Version: 1.15
- Description: Jupyter Notebook to automate the execution of web scans

In [ ]:
target = "www.bing.com"
top_target = "bing.com"
ffuf_subdomain_target = "FUZZ.bing.com"
keyword = "microsoft"
### Target is considered HTTPS by default, set True here if HTTP
target_http = False
### Change port if necessary
port = 443
### Set to 0 if scanning a subdomain
subdomain_recon = 1
### Cookie for target
### Example: "my_cookie=my_value; Path=/, other_cookie=other_value; Path=/test"
cookies = "test=test"

###
### No need to modify from here
###
import socket
import subprocess
import os

# URL is HTTP or HTTPS?
target_scheme = None

if target_http == True:
    print("HTTP site")
    target_http = True
    target_https = False
    target_scheme = "http://"
    target_url = "http://" + target + ":" + str(port) + "/"
else:
    print("HTTPS site")
    target_http = False
    target_https = True
    target_scheme = "https://"
    target_url = "https://" + target + ":" + str(port) + "/"

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

target_ip = socket.gethostbyname(target) 
ffuf_subdomain_target_https = "https://" + ffuf_subdomain_target
ffuf_subdomain_target_http = "http://" + ffuf_subdomain_target
ffuf_directory_brute = target + ":" + str(port) + "/FUZZ"
http_target = "http://" + target
https_target = "https://" + target

!mkdir -p tools/amass
!mkdir -p tools/arachni
!mkdir -p tools/ctfr
!mkdir -p tools/ffuf_subdomain
!mkdir -p tools/ffuf_directory
!mkdir -p tools/finalrecon
!mkdir -p tools/gospider
!mkdir -p tools/hakrawler
!mkdir -p tools/knockpy
!mkdir -p tools/nettacker
!mkdir -p tools/nikto
!mkdir -p tools/photon
!mkdir -p tools/puppeteer
!mkdir -p tools/puredns
!mkdir -p tools/subfinder
!mkdir -p tools/sn1per
!mkdir -p tools/sudomy
!mkdir -p tools/testssl
!mkdir -p tools/theharvester

!echo "1.1.1.1" > "`pwd`/tools/resolver.txt"

def read_file(location):
    try:
        with open(location) as f:
            text = f.read()
            return text
    except:
        return False

!pwd


# Target

In [ ]:
print(target)
print(target_ip)
print(target_url)


## Whois

In [ ]:
!dig {target}


# Pentester Details

## IP Address

In [ ]:
!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Scans

## spk

In [ ]:
!time docker run -it --rm redgo spk -silent -s "$keyword"


## metabigor

In [ ]:
!time docker run --rm redgo /bin/bash -c "echo $keyword | metabigor net --org"


## HTTP Headers

In [ ]:
%%bash -s "$target_url"
time curl -k -s --connect-timeout 10 -I $1


## shcheck

In [ ]:
!time docker run --rm shcheck -d {target_url}


## DrHeader

In [ ]:
!time docker run --entrypoint drheader --rm drheader scan --verify False single {target_url}


## ctfr

In [ ]:
!time docker run --rm -v "`pwd`/tools/ctfr/:/mnt" unapibageek/ctfr -d $top_target -o /mnt/results.txt


## Corsy

In [ ]:
!time python "/home/user/git/pentest-tools/Corsy/corsy.py" -u {target_url}


## waybackpy

In [ ]:
!time docker run --rm secsi/waybackpy --url {target} --oldest


## getjs

In [ ]:
!time docker run --rm secsi/getjs --insecure --url {target_url}


## httpx

In [ ]:
!time echo {target}:{port} | httpx -silent -status-code -title -tech-detect -ip -asn


## Gospider

In [ ]:
%%bash -s "$target_url" "$cookies"
time docker run -v "`pwd`/tools/gospider:/mnt" --rm secsi/gospider -o /mnt/ --cookie "$2" -s $1


## CMSeek

In [ ]:
!time docker run --rm secsi/cmseek --batch -u {target_url}


## WhatWeb

In [ ]:
%%bash -s "$target_url"
time docker run --rm guidelacour/whatweb ./whatweb $1 | sed -r "s/\x1B\[([0-9]{1,3}(;[0-9]{1,2})?)?[mGK]//g"


## hakrawler

In [ ]:
!time echo {target_url} | hakrawler -insecure | sort | uniq > "`pwd`/tools/hakrawler/results.txt"
!cat "`pwd`/tools/hakrawler/results.txt"


## Subfinder

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"
if [[ "$1" == "1" ]]; then
    # Scan
    time subfinder -d $2 -silent -active -oI -oJ -o tools/subfinder/results.txt 2>&1 >/dev/null
    # Output
    cat tools/subfinder/results.txt | jq -r '"Domain:  \(.host)\nIP addr: \(.ip)\n"'
else
    echo "Subdomain scans have been disabled."
fi


## Backup File Artifacts Checker

In [ ]:
!time docker run --rm secsi/bfac --no-text --url {target_url}


## cariddi

In [ ]:
%%bash -s "$target_url" "$cookies"
COOKIE_MOD=`echo $2 | sed 's/=/:/g'`
echo $1 | cariddi -headers "Cookie: $COOKIE_MOD" -e -s -info -err


## Screenshot

In [ ]:
import shutil
from pathlib import Path
home = str(Path.home())

os.system("docker run -u root --shm-size 1G --rm -v \"" + os.getcwd() + "/tools/puppeteer:/screenshots\" alekzonder/puppeteer:latest screenshot " + target_url + " 1024x768")
if not os.path.isfile("tools/puppeteer/screenshot_1024_768.png"):
    shutil.copyfile(home + "/git/maxos/resources/404.png", "tools/puppeteer/screenshot_1024_768.png")


![Target screenshot](tools/puppeteer/screenshot_1024_768.png)


## Email Enumeration

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"
if [[ "$1" == "1" ]]; then
    time docker run --rm infoga --domain $2 --source all --breach -v 2
else
    echo "Email enumeration disabled."
fi


## amass

In [ ]:
%%bash -s "$subdomain_recon" "$top_target" "$port"
if [[ "$1" == "1" ]]; then
    time amass enum -silent -nocolor -active -json tools/amass/results.txt -d $2 -p $3
    cat tools/amass/results.txt | jq -r '"##### \(.name)\nIP addr: \(.addresses|.[].ip)\nCIDR: \(.addresses|.[].cidr)\nASN: \(.addresses|.[].asn)\nDesc: \(.addresses|.[].desc)\n"'
else
    echo "DNS enumeration disabled"
fi


## ffuf (subdomains)

In [ ]:
%%bash -s "$subdomain_recon" "$ffuf_subdomain_target"

if [[ "$1" == "1" ]]; then
    echo {target_scheme}$2
    # Scan
    time ffuf -s -of all -w /home/user/git/wordlists/SecLists/Discovery/DNS/subdomains-top1million-5000.txt -u {target_scheme}$2 -od tools/ffuf_subdomain
    echo "`find tools/ffuf_subdomain -type f | wc -l` responses worth investigating.."
else
    echo "Subdomain scans have been disabled."
fi


## Knockpy

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"

if [[ "$1" == "1" ]]; then
    time docker run --rm -v "`pwd`/tools/knockpy:/mnt/" secsi/knockpy -o /mnt $2 2>&1 > /dev/null
    find tools/knockpy -type f -exec cat {} \;
else
    echo "Subdomain scans have been disabled."
fi


## Sudomy

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"

API_SHODAN=`cat /etc/api-shodan`
API_VIRUSTOTAL=`cat /etc/api-virustotal`

if [[ "$1" == "1" ]]; then
    time docker run -v "`pwd`/tools/sudomy:/usr/lib/sudomy/output" -e "SHODAN_API=$API_SHODAN" -e "VIRUSTOTAL=$API_VIRUSTOTAL" -t --rm screetsec/sudomy:v1.1.9-dev -d $2 -dP -eP -rS -cF -pS -tO -gW --httpx --dnsprobe  -aI webanalyze --slack -sS
else
    echo "Subdomain scans have been disabled."
fi


In [ ]:
!fd --full-path "`pwd`/tools/sudomy" --type f --extension lst --extension txt --extension out --size +0b --exec echo -e "\n\n###\n### {}\n###\n" \; --exec cat {}


## photon

In [ ]:
!time photon -c "{cookies}" --keys -u {target_url} -l 3 -t 100 --wayback -o tools/photon/{target}


### files.txt

In [ ]:
print(read_file("tools/photon/" + target + "/files.txt"))


### fuzzable.txt

In [ ]:
print(read_file("tools/photon/" + target + "/fuzzable.txt"))


### intel.txt

In [ ]:
print(read_file("tools/photon/" + target + "/intel.txt"))


### scripts.txt

In [ ]:
print(read_file("tools/photon/" + target + "/scripts.txt"))


### external.txt

In [ ]:
!time cat tools/photon/{target}/external.txt | httpx -silent -status-code -location -title -tech-detect -cl -ct -fc 401,403,404


### internal.txt

In [ ]:
!time cat tools/photon/{target}/internal.txt | httpx -silent -status-code -location -title -tech-detect -cl -ct -fc 401,403,404


## puredns

In [ ]:
!time docker run -it -v /home/user/wordlists/SecLists/Discovery/DNS:/mnt -v "`pwd`:/current" --rm secsi/puredns bruteforce /mnt/namelist.txt bir.ao --resolvers /current/tools/resolver.txt -q > "`pwd`/tools/puredns/results.txt"


In [ ]:
!cat "`pwd`/tools/puredns/results.txt"


## is-website-vulnerable

In [ ]:
!time docker run --rm lirantal/is-website-vulnerable:latest {target_url} --cookie "{cookies}"


## dvcs-ripper

In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-svn.pl -u {target_url}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-cvs.pl -u {target_url}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-bzr.pl -u {target_url}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-hg.pl -u {target_url}


In [ ]:
!time docker run --rm secsi/dvcs-ripper ./rip-git.pl -u {target_url}


## vulnx.py

In [ ]:
!time docker run -it --rm --name vulnx vulnx:latest -e -l all -w -d --dns -u {target_url}


## Dirhunt

In [ ]:
%%bash -s "$target_url" "$cookies"
COOKIE_MOD=`echo $2 | sed 's/=/:/g'`
time docker run --rm secsi/dirhunt -x not_found -c "$COOKIE_MOD" --progress-disabled $1 | grep -Ev "No interesting files detected"


## theHarvester

In [ ]:
!time docker run --rm -v "`pwd`/tools/theharvester:/mnt" simonthomas/theharvester:latest theharvester -d $top_target -b all -f /mnt/results.html


## dnscan

In [ ]:
%%bash -s "$subdomain_recon" "$target"

if [[ "$1" == "1" ]]; then
    time docker run --rm secsi/dnscan -d $2 -w subdomains-10000.txt -t 10 -R 1.0.0.1
else
    echo "Subdomain scans have been disabled."
fi


## fierce

In [ ]:
!time docker run --rm secsi/fierce --domain {target}


## dnstwist

In [ ]:
!time docker run --rm elceef/dnstwist {target}


## CloudFail

In [ ]:
!time docker run --rm secsi/cloudfail -u -t {target}


## sns

In [ ]:
!time docker run -it --rm redgo sns --check -u {target_url}


## Nuclei

In [ ]:
!time docker run --rm -v "`pwd`:/mnt" projectdiscovery/nuclei -H "Cookie:{cookies}" -u {target_url}


## TLS Scan

In [ ]:
%%bash -s "$target_https" "$target" "$port"

if [[ "$1" == "True" ]]; then
    time testssl.sh -oA tools/testssl $2:$3
fi


## Raccoon

In [ ]:
%%bash -s "$target_url" "$cookies"
COOKIE_MOD=`echo $2 | sed 's/=/:/g'`
time docker run --rm evyatarmeged/raccoon:latest $1 --skip-health-check -c "$COOKIE_MOD" --ignored-response-codes "301,302,400,401,403,404,503,504"


## ffuf (directory brute)


In [ ]:
!echo {target_scheme}{ffuf_directory_brute}
!time ffuf -b "$cookies" -r -s -ac -sf -of all -mc 200,204,301,302,307,405,500 -w /home/user/git/wordlists/SecLists/Discovery/Web-Content/raft-large-directories.txt -u {target_scheme}{ffuf_directory_brute} -od tools/ffuf_directory
!echo "`find tools/ffuf_directory -type f | wc -l` responses worth investigating.."
!find tools/ffuf_directory -type f -exec grep -hE 'GET |^HTTP/' {} \; -exec echo \;


## dnsenum

In [ ]:
%%bash -s "$subdomain_recon" "$top_target"

if [[ "$1" == "1" ]]; then
    time docker run --rm guidelacour/dnsenum ./dnsenum.pl $2 --file dns.txt
else
    echo "Subdomain scans have been disabled."
fi


## domain_analyzer

In [ ]:
!docker run --rm -it verovaleros/domain_analyzer:python2.7 /domain_analyzer/domain_analyzer.py -d {target}


## FinalRecon

In [ ]:
!time docker run --rm -v "`pwd`/tools/finalrecon:/root/.local/share/finalrecon/dumps/" thewhiteh4t/finalrecon {target_url} --full


In [ ]:
!find "`pwd`/tools/finalrecon/" -name "*.txt" -exec cat {} \;


## Nettacker

In [ ]:
!time docker run --rm -v "`pwd`/tools/nettacker:/usr/src/owaspnettacker/.data/results/" nettacker python3 ./nettacker.py -m all -i {target_url}


In [ ]:
!echo "Full Nettacker results in file://`pwd`/tools/nettacker"


## Arachni

In [ ]:
!time docker run --rm --net host -v "`pwd`/tools/arachni:/mnt" arachni/arachni:latest /usr/local/arachni/bin/arachni --http-cookie-string "$cookies" --timeout "0:30:0" --output-only-positives --report-save-path /mnt {target_url}


In [ ]:
!time docker run --rm --net host -v "`pwd`/tools/arachni:/mnt" arachni/arachni:latest find /mnt -type f -exec /usr/local/arachni/bin/arachni_reporter --reporter html:outfile=/mnt/arachni.zip {} \;


In [ ]:
!sudo chown user:users tools/arachni/arachni.zip
!unzip -o tools/arachni/arachni.zip -d tools/arachni/html_report


In [ ]:
!echo Results in file://`pwd`/tools/arachni/html_report/index.html


## Nikto

In [ ]:
!time docker run --rm --net=host -v "`pwd`/tools/nikto:/mnt/" sullo/nikto -ask No -nointeractive -Format json -output /mnt/nikto -h {target_url}


## dirsearch

In [ ]:
!time docker run --rm secsi/dirsearch --cookie="$cookies" -i 100-199,200-299 -q --no-color -F -u {target_url}


# Manual

## Nessus

Check .zsh_history


## reNgine

In [ ]:
!echo d-rengine


https://localhost:8020/

## Spiderfoot

http://localhost:8060/

## OpenVAS

https://localhost:8030/login/login.html (admin / admin)

## sn1per

Not run automatically because of large output and slow scan time

In [ ]:
!echo docker run -v \"`pwd`/tools/sn1per:/usr/share/sniper/loot/workspace/\" --rm -it xer0dayz/sn1per sniper -t {target}


In [ ]:
!echo find \"$PWD/tools/sn1per/"*"/output\" -type f -exec cat {} \\\;


# Notify

In [ ]:
!/home/user/git/maxos/scripts/telegram_notify.sh -a -m "PT Web Assessment complete for {target}."


# End Time

In [ ]:
!date
